# SNR of Sinusoid Plus Noise

This Jupyter notebook consists of an audio demonstration in <a href=#section1>Section 1</a> of a pure tone (sinusoid) sounds in the presence of addititive low-pass filtered white Gaussian noise (WGN); it aims to show how the pure tone's audibility depends on the Signal-to-Noise Ratio (SNR). To use this notebook, run all cells and hop from demo to demo.

**Notes:**
 - Requires Python 3.x; tested on Python 3.7.6
 - Requires `numpy`, `matplotlib`, `ipywidgets`, `IPython.display`.
 - It reuses the `PSD_BPF_WGN()` and `simRAPR()` functions from the `GP Sample Paths` Jupyter notebook. 
 - To install `ipywidgets`, use
    - `conda install ipywidgets` or `conda install -c conda-forge ipywidgets`
 - or
    - `pip install ipywidgets`
    - `jupyter nbextension enable --py --sys-prefix widgetsnbextension`


Authored by Georgios C. Anagnostopoulos 
ver. 1.0 (April 2020)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import Audio, display

import functools

In [2]:
# Some global settings for figure sizes
normalFigSize = (8, 6) # (width,height) in inches
largeFigSize = (12, 9)
xlargeFigSize = (18, 12)

## 0. Functions from `GP Sample Paths` Jupyter notebook

In [3]:
# PSD of BP-filtered WGN
# sigmaSquared>0: intensity of WGN
# omega_1>=0: first cut-off angular frequency of BPF in rad/sec
# omega_2>0: second cut-off angular frequency of BPF in rad/sec; must be > omega_1
def PSD_BPF_WGN(omega, sigmaSquared, omega_1, omega_2):
    absOmega = np.abs(omega) # PSD is an even function
    return sigmaSquared * np.less_equal(omega_1, absOmega) * np.less_equal(absOmega, omega_2) 


In [4]:
# Implementation of the RAPR simulation method for bandlimited,
# WSS, zero-mean GPs.
#
# t_min, t_max: time range in seconds for generating the sample path of the desired GP; must have t_min < t_max.
# fs_Hz>0: (time-domain) sampling rate (frequency) in Hz. Normally, should be > 2 * f_max_Hz.
# PSDfunc: a function that receives an omega array in rad/sec and returns the value of the desired GP's PSD.
# f_max_Hz>0: bandwidth of provided PSD in Hz.
def simRAPR(t_min, t_max, fs_Hz, PSDfunc, f_max_Hz):
    omega_B = 2.0 * np.pi * f_max_Hz # bandwidth in rad/sec     
    step = 1 / fs_Hz # sec
    t = np.arange(t_min, t_max, step, dtype=float) 

    # To avoid displaying more than one period of the sample path in [t_min, t_max],
    # one must choose N larger than frequency_2 * (t_max - t_min). 
    # Obviously, the larger N, the better the quality of the sample path.
    # The default value is 1000.
    N = int(max(999, f_max_Hz * (t_max - t_min))+1) 
    DeltaOmega = omega_B / N
    k = np.arange(0, N)
    omega = k * DeltaOmega    
    
    z_A = np.random.normal(0, 1, N) # sample a first set of i.i.d. standard Gaussian RVs
    z_B = np.random.normal(0, 1, N) # sample a second set of i.i.d. standard Gaussian RVs
    
    PSD = PSDfunc(omega) # sample the desired PSD
    
    A = np.sqrt(PSD * DeltaOmega / np.pi) * z_A
    B = np.sqrt(PSD * DeltaOmega / np.pi) * z_B
    
    omega_t = np.outer(t, omega)
    x = np.dot(np.cos(omega_t), A) + np.dot(np.sin(omega_t), B)
    return t, x

<a name='section1' /></a>

## 1. SNR of Pure Tone plus Low-Pass Filtered WGN

In communication systems, the Signal-to-Noise Ratio (SNR) at the receiver largely determines the recovery quality of the transmitted message. 

Here we'll consider that the received signal (actually, it is a Random Process (RP)) $Y$ consists of a pure tone (a single sinusoid; the message) plus low-pass filtered White Gaussian Noise (the noise). 

The role of the message is played by the (deterministic) sinusoid

\begin{align*}
    m(t) \triangleq A \cos(\omega_o t)
\end{align*}

for some $A, \omega_o > 0$, whose avergae power is computed as $P_m = \frac{A^2}{2}$. On the other hand, the additive noise is a zero-mean, wisde-sense stationary (WSS) Gaussian Process (GP) $N$ with Power Spectral Density (PSD) 

\begin{align*}
    S_N(j 2 \pi f) \triangleq \sigma^2 \ [|f| \leq 2 \Delta f_B]
\end{align*}

for some intensity $\sigma^2 > 0$ and some bandwidth $2 \Delta f_B > 0$. The average power of $N$ is given as $P_N = 4 \sigma^2 \Delta f_B$. Then, the SNR is given as

\begin{align*}
    \mathrm{SNR} \triangleq \frac{P_m}{P_N} = \frac{A^2}{8 \sigma^2 \Delta f_B}
\end{align*}

or, in decibels, as

\begin{align*}
    \mathrm{SNR}_{dB} \triangleq 10 \log_{10}(SNR)
\end{align*}

By simulating the RP $Y(t) \triangleq m(t) + N(t)$, in the following audio demonstration we'll show the effect of SNR on the audibility of the tone in the presense of the aformentioned noise. Towards this, we'll select $\Delta f_B = f_o \triangleq  \frac{2 \pi}{\omega_o}$ and .

It turns out that $Y$ will be a GP with a periodic mean $m(t)$ (with fundamental period $T_o \triangleq \frac{2 \pi}{\omega_o}$) and a cyclo-stationary auto-correlation $R_Y(t + \tau, t) = R_N(\tau) + m(t + \tau)m(t)$ with the same period. 

In [5]:
# Global variables/objects for this section
SinusoidFrequencyHz=1000.0 # Hz
samplingRateHz = 6000 # Hz #44100 # Hz; audio CD quality sampling rate 
durationSeconds = 3 # sec

SNRdBSlider = widgets.FloatSlider(
    value=60.0,
    min=-10.0,
    max=60.0,
    step=5.0,
    description='SNR (dB)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [6]:
def soundSinusoidalPlusNoise(SNRdB=SNRdBSlider):
    
    DfB = SinusoidFrequencyHz
    sigmaSquared = 1e-10
    SinusoidAmplitude = np.sqrt(8 * DfB * sigmaSquared) * 10.0**(SNRdB / 20.0)
    
    t = np.linspace(0.0, durationSeconds, samplingRateHz * durationSeconds)
    m = SinusoidAmplitude * np.cos(2 * np.pi * SinusoidFrequencyHz * t)   

    myPSDfunc = functools.partial(PSD_BPF_WGN, sigmaSquared=sigmaSquared, 
                                  omega_1 = 2.0 * np.pi * (SinusoidFrequencyHz - DfB / 2.0), 
                                  omega_2 = 2.0 * np.pi * (SinusoidFrequencyHz + DfB / 2.0))

    t, n = simRAPR(0.0, durationSeconds, samplingRateHz, myPSDfunc, 2.0 * SinusoidFrequencyHz)
    
    y = m + n
    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, y, lw=1.0, color='blue')
    ax.set_xlim(0.0, 0.05)
    #ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t (sec)$',fontsize=18)
    ax.set_ylabel('$Y(t)$', fontsize=18)
    #ax.grid(showGrid)    
    
    display(Audio(data=y, rate=samplingRateHz))
    return y


v = interactive(soundSinusoidalPlusNoise, {'manual': True})
display(v)

interactive(children=(FloatSlider(value=60.0, continuous_update=False, description='SNR (dB)', max=60.0, min=-…

**Instructions:**

* To get an approximate sample path and audio each time, click the `Run Interact` button.
* You can download the generated audio as a `WAV` file by clicking on the three vertical dots to the left of the audio widget and select `Download`. 

**Observations:**

Depending on the speakers one uses to listen to the sample path and on the listener's age among other things:

* For an SNR of 60 dB, we virually hear a pure tone at 1Khz.
* For an SNR of 50 dB, we start hearing a very faint hint of noise.
* As we decrease the SNR, the noise becomes more prominent.
* At an SNR of 5 dB, we hear the tone and the noise at roughly the same apparent loudness.
* At an SNR of -5 dB, we may hear the tone, but only faintly so.
* At an SNR of -10 dB, we do not hear a tone any more.
